In [12]:
import pandas as pd
import numpy as np
import ldtk

In [13]:
target_path = '/Users/u5500483/Documents/GitHub/Paper_II/'
filter_path = '/Users/u5500483/Documents/GitHub/Paper_II/filters_limb_dark/'

In [14]:
def create_ngts():
    df = pd.read_csv(filter_path + 'NGTS_transmission_and_atmosphere_MGunther.txt', comment='#', header=None, names=['wavelength','response'])
    return ldtk.TabulatedFilter('NGTS', df.wavelength.values, df.response.values)

def create_tess():
    df = pd.read_table(filter_path + 'TESS.dat', header=None, names=['wavelength','response'], sep='    ')
    df['wavelength'] = df['wavelength']*0.1
    return ldtk.TabulatedFilter('TESS', df.wavelength.values, df.response.values)

def create_SLOAN_g():
    df = pd.read_table(filter_path + 'SLOAN_SDSS.gprime_filter.dat', header=None, names=['wavelength','response'])
    df['wavelength'] = df['wavelength'].astype('float')*0.1
    return ldtk.TabulatedFilter('SLOAN-g', df.wavelength.values, df.response.values)


# def create_wasp():
#     df = pd.read_table('/Users/u5500483/Downloads/filters/WASP.dat', header=None, names=['wavelength','response'], delim_whitespace=True)
#     df['wavelength'] = df['wavelength'].astype('float')*0.1
#     return ldtk.TabulatedFilter('WASP', df.wavelength.values, df.response.values)
#
# def create_Johnson(filter):
#     df_path = '/Users/u5500483/Downloads/filters/Generic_Johnson.{}.dat'.format(filter)
#     df = pd.read_table(df_path, header=None, names=['wavelength','response'], sep=' ')
#     df['wavelength'] = df['wavelength']*0.1
#     return ldtk.TabulatedFilter(filter, df.wavelength.values, df.response.values)


In [15]:
ngts = create_ngts()
SLOAN_g = create_SLOAN_g()
# tess = create_tess()

# Limb darkening Coefficients

In [16]:
stellar_df = pd.read_csv(target_path+'stellar_params.dat', comment='#', names=['Parameter','median','upper','lower','3sig_low','3sig_up'], delim_whitespace=True)
display(stellar_df)

,Parameter,median,upper,lower,3sig_low,3sig_up
0,teff,6226.00000,162.250000,117.399000,NaN,NaN
1,logg,4.25879,0.107285,0.066733,NaN,NaN
2,[Fe/H],0.10700,0.080000,0.080000,NaN,NaN


In [17]:
filter_names = np.array(['NGTS', 'SLOAN-g'])
filters = np.array([ngts, SLOAN_g])
#Enter stellar parameters in form (median, 1-sig error). I pull these values from ARIADNE fits but catalogue values are fine
sc = ldtk.LDPSetCreator(teff=(stellar_df.loc[stellar_df.Parameter=='teff', 'median'].item(),np.max([stellar_df.loc[stellar_df.Parameter=='teff', 'upper'].item(), stellar_df.loc[stellar_df.Parameter=='teff', 'lower'].item()])),    # First col value, second col plusminus error
                  logg=(stellar_df.loc[stellar_df.Parameter=='logg', 'median'].item(),np.max([stellar_df.loc[stellar_df.Parameter=='logg', 'upper'].item(), stellar_df.loc[stellar_df.Parameter=='logg', 'lower'].item()])),
                      z=(stellar_df.loc[stellar_df.Parameter=='[Fe/H]', 'median'].item(),np.max([stellar_df.loc[stellar_df.Parameter=='[Fe/H]', 'upper'].item(), stellar_df.loc[stellar_df.Parameter=='[Fe/H]', 'lower'].item()])),
                     filters=filters)
ps = sc.create_profiles(nsamples=2000)                # Create the limb darkening profiles
ps.resample_linear_z(300)
cq,eq = ps.coeffs_qd(do_mc=True)         # Estimate quadratic law coefficients
for i, filter in enumerate(filter_names):
    print(filter)
    print('c1/c2 values:')
    print(cq[i,0],eq[i,0])
    print(cq[i,1],eq[i,1])
print('....')
for i, filter in enumerate(filter_names):
    print(filter)
    q1 = (cq[i,0]+cq[i,1])**2
    q1err = (eq[i,0]**2+eq[i,1]**2)**0.5/(cq[i,0]+cq[i,1])*2*q1
    q2 = cq[i,0]/(2*(cq[i,0]+cq[i,1]))
    q2err = q2*((eq[i,0]/cq[i,0])**2+((eq[i,0]**2+eq[i,1]**2)**0.5/(cq[i,0]+cq[i,1])))**0.5
    # print('q1={}+/-{}. 5-sig = {}'.format(q1,q1err,q1err*5))
    # print('q2={}+/-{}. 5-sig = {}'.format(q2,q2err,q2err*5))
    print("host_ldc_q1_"+str(filter)+","+str(q1)+",1,normal "+str(q1)+" "+str(q1err*3)+",$q_{1; \mathrm{"+str(filter)+"}}$,")
    print("host_ldc_q2_"+str(filter)+","+str(q2)+",1,normal "+str(q2)+" "+str(q2err*3)+",$q_{1; \mathrm{"+str(filter)+"}}$,")

Need to download 106 files, approximately 35.40 MB


LDTk downloading uncached files:   0%|          | 0/106 [00:00<?, ?it/s]

NGTS
c1/c2 values:
0.44629050513065993 0.0010589346070618841
0.14052696783447394 0.002833616090408942
SLOAN-g
c1/c2 values:
0.6491952297361383 0.0018193481972803517
0.1323432682522901 0.0038336203734956175
....
NGTS
host_ldc_q1_NGTS,0.3443547465771856,1,normal 0.3443547465771856 0.010650793963232775,$q_{1; \mathrm{NGTS}}$,
host_ldc_q2_NGTS,0.38026347688285056,1,normal 0.38026347688285056 0.08195118258892428,$q_{1; \mathrm{NGTS}}$,
SLOAN-g
host_ldc_q1_SLOAN-g,0.6108024238380086,1,normal 0.6108024238380086 0.01989840956393182,$q_{1; \mathrm{SLOAN-g}}$,
host_ldc_q2_SLOAN-g,0.41533157445671376,1,normal 0.41533157445671376 0.09187836509266895,$q_{1; \mathrm{SLOAN-g}}$,
